In [1]:
#default_exp oo_hp

In [2]:
#export
import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

In [3]:
#export
from copy import deepcopy

In [4]:
#export
import optuna

In [5]:
#export
SEED = 42

In [6]:
from nbdev import export

In [7]:
from nbdev.export import *

In [8]:
#export
from collections.abc import Iterable

# HyPSTER Classes

In [9]:
#export
from inspect import signature, Parameter
import functools

def auto_assign(func):
    # Signature:
    sig = signature(func)
    for name, param in sig.parameters.items():
        if param.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise RuntimeError('Unable to auto assign if *args or **kwargs in signature.')
    # Wrapper:
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        for i, (name, param) in enumerate(sig.parameters.items()):
            # Skip 'self' param:
            if i == 0: continue
            # Search value in args, kwargs or defaults:
            if i - 1 < len(args):
                val = args[i - 1]
            elif name in kwargs:
                val = kwargs[name]
            else:
                val = param.default
            setattr(self, name, val)
        func(self, *args, **kwargs)
    return wrapper

In [10]:
#export
class HypsterBase:
    def __init__(self): return
#TODO: add stuff here

In [11]:
#export
class HpExpression:
    def __init__(self, exp1, exp2):
        self.exp1 = exp1; self.exp2 = exp2
    
    def sample(self, trial): raise NotImplementedError
    
    #TODO: check what to do when name=None
    def get_name(self):
        if self.name is not None: return self.name

        name = ""
        if self.exp1 is not None and isinstance(self.exp1, HpExpression) and hasattr(self.exp1, "name"):
            name += self.exp1.name
        if self.exp2 is not None and isinstance(self.exp2, HpExpression) and hasattr(self.exp2, "name"):
            if len(name) > 0:
                name +=  "_"
            name += self.exp2.name
        self.name = name
        return self.name
        #TODO: refactor

    def __add__(self, other):  return AddExpression(self, other)
    def __radd__(self, other): return AddExpression(other, self)
    def __sub__(self, other):  return SubExpression(self, other)
    def __rsub__(self, other): return SubExpression(other, self)
    def __mul__(self, other):  return MulExpression(self, other)
    def __rmul__(self, other): return MulExpression(other, self)
    def __div__(self, other):  return DivExpression(self, other)
    def __rdiv__(self, other): return DivExpression(other, self)
    def __pow__(self, other):  return PowExpression(self, other)
    def __rpow__(self, other): return PowExpression(other, self)

In [12]:
#export
HYPSTER_TYPES = (HypsterBase, HpExpression)

In [13]:
#export
class SubExpression(HpExpression):
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 - exp2

In [14]:
#export
class AddExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 + exp2

In [15]:
#export
class MulExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 * exp2

In [16]:
#export
class DivExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 / exp2

In [17]:
#export
class PowExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 ** exp2

In [18]:
#TODO: add round, int, floor etc...
#TODO: add Brackets (), [] etc...?

In [19]:
def foo(a, b="hello"):
    print(a)

In [20]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

In [21]:
def get_name_from_hp(hp):
    return remove_prefix(hp.__class__.__name__.lower(), "hp")

In [22]:
# upon init -> assign a name by manual name or type
# upon using prepare - if there isn't a manual name - assign the name by arg_name

In [23]:
# upon initializing a study, recursively go through arguments and check if there are same expressions.
# for each arg in args:
    # if there is another name and both manually defined -- error
    # if there is another name and both and not manually defined --> add suffix to both by order
    # if one is not manually defined --> get serial number of current and add suffix only to the one not manually defined

In [24]:
#export
def set_init_name(default_name, name):
    if name is not None:
        return name, True
    else:
        return default_name, False

In [25]:
#export
class HpInt(HpExpression):
    @auto_assign
    def __init__(self, low, high, step=1, name=None):
        self.name, self.manual_name = set_init_name("int", name)
    
    def sample(self, trial):
        return trial.suggest_int(self.name, self.low, self.high, self.step)

In [26]:
#export
class HpFloat(HpExpression):
    @auto_assign
    def __init__(self, low, high, log=False, step=None, name=None):
        self.name, self.manual_name = set_init_name("float", name)
        #TODO: check what's up with log and step
        #TODO: move result to HpExpression?
    
    def sample(self, trial):
        return trial.suggest_float(self.name, self.low, self.high)
    
    #TODO: warn if log=True & step is not None
    #TODO: check what is the "*" in the function definition

In [27]:
#export
def _log_optuna_param(param_name, result, trial):
    trial.set_user_attr(param_name, result)

In [28]:
#export
class HpFunc(HpExpression):
    def __init__(self, func, name=None, **kwargs):
        self.name, self.manual_name = set_init_name("func", name)
        self.func = func
        self.kwargs = kwargs
        
    def sample(self, trial):
        result = self.func(trial, **self.kwargs)
        _log_optuna_param(self.name, result, trial)
        return result

In [29]:
#export
class HpCategorical(HpExpression):
    @auto_assign
    def __init__(self, choices, name=None):
        self.name, self.manual_name = set_init_name("categorical", name)

    def sample(self, trial):         
        choices           = self.choices
        name              = self.name
        optuna_valid_cats = [str, int, float, bool, NoneType] #TODO: add more + move to global area
        
        if any([type(choice) not in optuna_valid_cats for choice in self.choices]):
            #TODO: add check for "choice.__name__"
            self.items_str = [choice.__name__ for choice in choices]
            self.str_dict  = dict(zip(self.items_str, choices))
            chosen_hp      = trial.suggest_categorical(name, self.str_dict)
            result         = self.str_dict[chosen_hp]
        else:
            result = trial.suggest_categorical(name, choices)
        return result

In [30]:
#export
class HpBool(HpCategorical):
    def __init__(self, name=None):
        super().__init__(choices=[False, True])
        self.name, self.manual_name = set_init_name("boolean", name)

In [31]:
#export
class HpToggle(HpBool):
    @auto_assign
    def __init__(self, hp, name=None):
        if name is None:
            self.manual_name = False
            if isinstance(hp, HpExpression) and hp.name is not None:
                self.name = f"toggle_{hp.name}"
                
            elif hasattr(hp, "__class__") and hasattr(hp.__class__, "__name__"):
                self.name = f"toggle_{hp.__class__.__name__}"
            else:
                self.name = "toggle"
        else:
            self.name = name
            self.manual_name = True
        
    def sample(self, trial): return HpBool(self.name).sample(trial)

In [32]:
#export
class HpVarLenList(HpExpression):
    #TODO: think of a better name?
    @auto_assign
    def __init__(self, min_len, max_len, hp, same_value=False, name=None):
        self.name, self.manual_name = set_init_name("var_len_list", name)
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not contains_hypster(self.hp, HYPSTER_TYPES)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.result = None
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)

        return lst

In [33]:
#TODO: make HpVarLenTuple

In [34]:
#export
class HpIterable:
    def __init__(self): return

In [35]:
#export
def list_to_tuple(lst): return (*lst, )

In [36]:
#export
def populate_iterable(iterable, trial):
    sampled_lst = []
    for item in iterable:
        if isinstance(item, HpToggle):
            if sample_hp(item, trial):
                sampled_lst.append(sample_hp(item.hp, trial))
        else:
            sampled_lst.append(sample_hp(item, trial))
    return sampled_lst

In [37]:
#export
def populate_dict(dct, trial):
    sampled_dict = {}
    for key, value in dct.items():
        if isinstance(value, HpToggle):
            if sample_hp(value, trial):
                sampled_dict[key] = sample_hp(value.hp, trial)
        else:
            sampled_dict[key] = sample_hp(value, trial)
    return sampled_dict

In [38]:
#export
class HpList(HpIterable):
    @auto_assign
    def __init__(self, lst): pass
    
    def sample(self, trial):
        return populate_iterable(self.lst, trial)

In [39]:
#export
class HpTuple(HpIterable):
    @auto_assign
    def __init__(self, tup): pass
    
    def sample(self, trial):
        return list_to_tuple(populate_iterable(self.tup, trial))

In [40]:
#export
class HpDict(HpIterable):
    @auto_assign
    def __init__(self, dct): pass
    
    def sample(self, trial):
        return populate_dict(self.dct, trial)

# Tests

In [41]:
#export
DATA_STRUCTURES = (set, list, tuple, dict)

In [42]:
#export
def contains_hypster(x, types):
    if not isinstance(x, DATA_STRUCTURES):
        x = [x]
    elif isinstance(x, dict):
        x = x.values()
        
    for item in x:
        if isinstance(item, DATA_STRUCTURES):
            if contains_hypster(item, types):
                return True
        else:
            if isinstance(item, types):
                return True
    return False

In [43]:
#TODO!: check if class attributes / methods? have hypster in them

## Test contains_hypster

In [44]:
hps = []

In [45]:
hps.append(HpInt(2, 10))

In [46]:
hps.append({"hi" : 4, "hello": HpFloat(1.0, 2.0)})

In [47]:
hps.append(["a", 2, HpInt(1, 19)])

In [48]:
hps.append(["a", 2, {"ho": "hello"}, {"hi" : HpCategorical([1,2,3])}])

In [49]:
[contains_hypster(hp, HpExpression) for hp in hps]

[True, True, True, True]

In [50]:
#export
def sample_hp(hp, trial):
    if not contains_hypster(hp, HYPSTER_TYPES):
        return hp
    #TODO: change to dynamic dispatch
    if isinstance(hp, list):
        hp = HpList(hp)
    elif isinstance(hp, tuple):
        hp = HpTuple(hp)
    elif isinstance(hp, dict):
        hp = HpDict(hp)
    return hp.sample(trial)
#TODO: handle list of lists
#TODO!: check if class attributes / methods? have hypster in them

In [51]:
#export
def run_hp_test(hp):
    def objective(trial):
        print(sample_hp(hp, trial))
        return 1.0        
    
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=5, timeout=600)
    return study

### HpToggle

In [52]:
x = [1, 5, HpToggle(10)]

In [53]:
study = run_hp_test(x)

[1, 5]
[1, 5, 10]
[1, 5, 10]
[1, 5]
[1, 5, 10]


In [54]:
y = (1, 5, HpToggle(10))

In [55]:
run_hp_test(y)

(1, 5, 10)
(1, 5, 10)
(1, 5, 10)
(1, 5)
(1, 5)


In [56]:
z = {"hi" : 1, "hello" : 5, "howdy" : HpToggle(10)}

In [57]:
run_hp_test(z)

{'hi': 1, 'hello': 5, 'howdy': 10}
{'hi': 1, 'hello': 5, 'howdy': 10}
{'hi': 1, 'hello': 5}
{'hi': 1, 'hello': 5, 'howdy': 10}
{'hi': 1, 'hello': 5}


### Int

In [58]:
x = HpInt(2, 10)
x_lst = [HpInt(2, 10, name="int_1"), HpInt(12,29, name="int_2")]
y = HpInt(50, 300, 50)

In [59]:
study = run_hp_test(x)

2
7
6
5
6


In [60]:
study = run_hp_test(x_lst)

[3, 21]
[2, 29]
[4, 24]
[7, 25]
[3, 29]


In [61]:
run_hp_test(y)

300
200
300
200
250


In [62]:
z = HpVarLenList(1, 5, x, same_value=False)

In [63]:
run_hp_test(z)

[7, 9, 5, 3, 2]
[2, 5, 7]
[10, 3, 6, 10, 4]
[5]
[8, 7, 5]


### Float

In [64]:
x = HpFloat(1.0, 2.0)
y = HpVarLenList(1, 5, x, same_value=False)

In [65]:
run_hp_test(x)

1.180864872308181
1.2662859764184922
1.1443112633667802
1.9436805724550816
1.772682914217706


In [66]:
run_hp_test(y)

[1.5058304630792991]
[1.1626948626617564, 1.793465423960677, 1.1475072149268832]
[1.4980978212773792, 1.9622718144985607]
[1.4598367269022274, 1.5010161812476452, 1.0593921942099505]
[1.2405179523631638, 1.5967285276312924, 1.3852228277542706, 1.6706037479420281, 1.1575539020429049]


### Categorical

In [67]:
x = HpCategorical(["cat", "meow", "rrrrrr"])
y = HpCategorical([Adam, SGD, QHAdam])
z = HpVarLenList(1, 5, x, same_value=False)

In [68]:
run_hp_test(x)

rrrrrr
rrrrrr
rrrrrr
meow
meow


In [69]:
run_hp_test(y)

<function QHAdam at 0x00000266E70E4268>
<function SGD at 0x00000266E70E0A60>
<function QHAdam at 0x00000266E70E4268>
<function QHAdam at 0x00000266E70E4268>
<function SGD at 0x00000266E70E0A60>


In [70]:
run_hp_test(z)

['cat', 'meow', 'rrrrrr', 'cat']
['cat', 'cat']
['rrrrrr', 'rrrrrr', 'meow', 'rrrrrr']
['meow', 'meow', 'cat', 'rrrrrr', 'cat']
['meow', 'meow', 'cat']


### Boolean

In [71]:
x = HpBool()
y = HpVarLenList(1, 5, x, same_value=False)

In [72]:
run_hp_test(x)

False
True
False
True
False


In [73]:
run_hp_test(y)

[True, False]
[True, False, True, False, True]
[True, True, False]
[False]
[True, False, True, True, True]


### HpIterable

In [74]:
x = HpBool()

In [75]:
lst = [x, 5]
tup = [x, 5]
dct = {"first" : x, "second" : 5}

In [76]:
start_mom = HpFloat(0.85, 0.99)

In [77]:
mom = (start_mom, start_mom - 0.1)

In [78]:
run_hp_test(mom)

(0.8760376548396693, 0.7760376548396694)
(0.9174187462512133, 0.8174187462512134)
(0.9449105309508516, 0.8449105309508517)
(0.8821444596736566, 0.7821444596736566)
(0.9858478870019507, 0.8858478870019507)


In [79]:
run_hp_test(lst)

[True, 5]
[False, 5]
[True, 5]
[False, 5]
[True, 5]


In [80]:
run_hp_test(tup)

[False, 5]
[False, 5]
[False, 5]
[True, 5]
[False, 5]


In [81]:
run_hp_test(dct)

{'first': False, 'second': 5}
{'first': True, 'second': 5}
{'first': True, 'second': 5}
{'first': True, 'second': 5}
{'first': True, 'second': 5}


In [82]:
#TODO consider adding "name" into HpExpression

In [83]:
#HpFunc

In [84]:
#TODO: support expressions like HpInt(...) * HpFloat(...) ?

In [85]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
